In [3]:
import requests
from bs4 import BeautifulSoup

baseUrl = 'https://www.wine.com'

page = requests.get(f'{baseUrl}/list/wine/red-wine/7155-124?showOutOfStock=true')

soup = BeautifulSoup(page.text, 'html.parser')


In [4]:
wineList = soup.find('ul', class_='listGridLayout_list')
wineElements = wineList.find_all('li', class_='listGridLayout_listItem')

In [25]:
for wineElem in wineElements[:3]:
    wineInfo = { 'color': 'red' }
    a = wineElem.find('a', class_='event_productClick', href=True)
    
    wineUrl = f"{baseUrl}{a['href']}"
    print(wineUrl)

    winePage = requests.get(wineUrl)
    wineSoup = BeautifulSoup(winePage.text, 'html.parser')

    # Name and vintage  
    name = wineSoup.find('h1', class_='pipName').text
    vintage = name[-4:]
    name = name[:-4].strip()
    wineInfo['name'] = name
    wineInfo['vintage'] = vintage

    # Varietal and Origin
    divOrigin = wineSoup.find('div', class_='pipOrigin')

    spanVarietal = divOrigin.find('span', class_='prodItemInfo_varietal')
    varietal = spanVarietal.find('a', href=True).text
    wineInfo['varietal'] = varietal

    spanOrigin = divOrigin.find('span', class_='prodItemInfo_originText')
    origin = spanOrigin.find('a', href=True).text
    wineInfo['origin'] = origin

    priceDiv = wineSoup.find('div', class_='productPrice_price-sale')
    if priceDiv:
        price = priceDiv.find('span', class_='productPrice_price-saleWhole').text.strip()
        price += "." + priceDiv.find('span', class_='productPrice_price-saleFractional').text.strip()
        wineInfo['price'] = float(price)

    # Ratings
    ulRatings = wineSoup.find('ul', class_='pipProdWineRatings')
    if ulRatings:
        ratings = []
        ratingsList = ulRatings.find_all('li', class_='pipProdWineRatings_item')
        for ratingElem in ratingsList:
            ratingName = ratingElem.contents[2].strip() + " " + ratingElem.contents[4].strip()
            
            ratingValue = ratingElem.find('span', class_='pipProdWineRatings_rating').text

            rating = {'name': ratingName, 'rating': int(ratingValue)}

            ratings.append(rating)

        wineInfo['ratings'] = ratings

    productDetails = wineSoup.find('section', class_='pipProdDetails')
    if productDetails:
        titles = productDetails.find_all('div', class_='pipProdDetails_title')
        values = productDetails.find_all('div', class_='pipProdDetails_name')
        for i, title in enumerate(titles):
            if title.text.strip() == 'Size':
                wineInfo['size'] = values[i].text[:-2]
            elif title.text.strip() == 'Producer':
                wineInfo['producer'] = values[i].text
                wineInfo['name'] = wineInfo['name'].replace(wineInfo['producer'], '').strip()
    else:
        size = wineSoup.find('span', class_='prodAlcoholVolume_text').text
        wineInfo['size'] = size

    divImage = wineSoup.find('div', class_='pipProdThumbs')
    images = []
    for imageSpan in divImage.find_all('span', class_='pipProdThumb'):
        images.append(imageSpan.find('img')['src'])
    if len(images):
        wineInfo['image'] = images

    for key, value in wineInfo.items():
        print(f"{key}: {value}")
    print()


https://www.wine.com/product/antinori-badia-a-passignano-chianti-classico-gran-selezione-2018/806034


AttributeError: 'NoneType' object has no attribute 'find_all'

In [33]:
text = 'Antinori Badia a Passignano Chianti Classico Gran Selezione 2018'

print(text.replace('Antinori', ''))

 Badia a Passignano Chianti Classico Gran Selezione 2018
